In [0]:
import numpy as np
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
import random
import sys
import os

#Setting basic strain descriptions

##From ABGRSS

In [0]:
df_kushy = pd.read_csv("https://raw.githubusercontent.com/abgrss/Cannabis-NLP/master/df_sentiment_analysis.csv")

In [88]:
df_kushy['Strain_description'][0]

'$100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant, $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert, cerebral feeling thanks to its sativa side.'

In [89]:
df_kushy.head()

,Strain,Strain.1,Cluster,Lean,Rating,Strain_type,Strain_description,Sentiment,Subjectivity
0,$100 OG,$100 OG,0,0.50,3.980556,hybrid,$100 OG is a 50/50 hybrid strain that packs a ...,0.1641,0.4269
1,'98 Aloha White Widow,'98 Aloha White Widow,2,0.57,4.666667,hybrid,The 98 Aloha White Widow is an especially pot...,0.2031,0.3187
2,1024,1024,0,0.30,4.422222,sativa,1024 is a sativa-dominant hybrid bred in Spain...,-0.0726,0.3929
3,13 Dawgs,13 Dawgs,0,0.50,4.000000,hybrid,13 Dawgs is a hybrid of G13 and Chemdawg genet...,0.5375,0.7875
4,22,22,0,0.54,3.857143,hybrid,22 is a Jack Herer cross from Cali Connection....,0.0306,0.4319


In [0]:
df_kushy = df_kushy.drop(["Rating", "Strain.1", "Cluster", "Lean", "Rating", "Sentiment", "Subjectivity"], axis=1)

In [91]:
df_kushy.head()

,Strain,Strain_type,Strain_description
0,$100 OG,hybrid,$100 OG is a 50/50 hybrid strain that packs a ...
1,'98 Aloha White Widow,hybrid,The 98 Aloha White Widow is an especially pot...
2,1024,sativa,1024 is a sativa-dominant hybrid bred in Spain...
3,13 Dawgs,hybrid,13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,22,hybrid,22 is a Jack Herer cross from Cali Connection....


This pushes descriptions to a file to be added to the /data/clean on repo.

In [0]:
df_kushy.to_csv('strain_descriptions.csv')

In [93]:
len(df_kushy)

2130

In [0]:
text_string = []

In [95]:
type(df_kushy['Strain_description'])

pandas.core.series.Series

In [96]:
df_kushy.isnull().count()

Strain                2130
Strain_type           2130
Strain_description    2130
dtype: int64

In [0]:
for each in range(2129):
  #string = df_kushy['Strain_description'][each]
  text_string = text_string + text_to_word_sequence(str(df_kushy['Strain_description'][each]))
  text_string = text_string + text_to_word_sequence(str(df_kushy['Strain_type'][each]))
  text_string = text_string + text_to_word_sequence(str(df_kushy['Strain'][each]))



In [0]:
text_string = text_to_word_sequence(df_kushy['Strain_description'][2129]) + text_string

In [99]:
text_string

['alenuihaha',
 'is',
 'a',
 'sativa',
 'dominant',
 'hybrid',
 'by',
 'pua',
 'mana',
 '1st',
 'hawaiian',
 'pakalolo',
 'seed',
 'bank',
 'named',
 'after',
 'the',
 'sea',
 'channel',
 'that',
 'separates',
 'the',
 'kona',
 'side',
 'of',
 'the',
 'big',
 'island',
 'and',
 'the',
 'island',
 'of',
 'maui',
 'by',
 'blending',
 'kona',
 'gold',
 'and',
 'maui',
 'wowie',
 'alenuihaha',
 'achieves',
 'an',
 'uplifted',
 'and',
 'happy',
 'stimulation',
 'that',
 "doesn't",
 'overclock',
 'the',
 "consumer's",
 'heart',
 'or',
 'mind',
 'this',
 "strain's",
 'healthy',
 'yield',
 'and',
 'consistently',
 'positive',
 'effects',
 'make',
 'it',
 'a',
 'delightful',
 'and',
 'inspiring',
 'choice',
 'for',
 'consumers',
 'seeking',
 'relief',
 'from',
 'depression',
 'and',
 'fatigue',
 '100',
 'og',
 'is',
 'a',
 '50',
 '50',
 'hybrid',
 'strain',
 'that',
 'packs',
 'a',
 'strong',
 'punch',
 'the',
 'name',
 'supposedly',
 'refers',
 'to',
 'both',
 'its',
 'strength',
 'and',
 'hig

In [100]:
len(text_string)

165543

In [0]:
t = Tokenizer()
t.fit_on_texts(text_string)

In [102]:
print(t.word_counts)

OrderedDict([('alenuihaha', 2), ('is', 3487), ('a', 5914), ('sativa', 1410), ('dominant', 854), ('hybrid', 2246), ('by', 1392), ('pua', 12), ('mana', 11), ('1st', 54), ('hawaiian', 117), ('pakalolo', 10), ('seed', 130), ('bank', 31), ('named', 150), ('after', 240), ('the', 5317), ('sea', 13), ('channel', 1), ('that', 2059), ('separates', 1), ('kona', 14), ('side', 69), ('of', 4215), ('big', 90), ('island', 38), ('and', 6641), ('maui', 48), ('blending', 10), ('gold', 84), ('wowie', 14), ('achieves', 7), ('an', 1158), ('uplifted', 30), ('happy', 127), ('stimulation', 54), ("doesn't", 14), ('overclock', 1), ("consumer's", 14), ('heart', 13), ('or', 420), ('mind', 294), ('this', 2189), ("strain's", 162), ('healthy', 16), ('yield', 82), ('consistently', 9), ('positive', 15), ('effects', 1327), ('make', 159), ('it', 758), ('delightful', 8), ('inspiring', 7), ('choice', 152), ('for', 1631), ('consumers', 276), ('seeking', 57), ('relief', 196), ('from', 947), ('depression', 150), ('fatigue', 4

In [103]:
print(t.document_count)

165543


In [104]:
print(t.word_index)

{'and': 1, 'a': 2, 'the': 3, 'of': 4, 'is': 5, 'with': 6, 'to': 7, 'hybrid': 8, 'strain': 9, 'this': 10, 'that': 11, 'indica': 12, 'in': 13, 'for': 14, 'its': 15, 'sativa': 16, 'by': 17, 'effects': 18, 'an': 19, 'og': 20, 'kush': 21, 'from': 22, 'as': 23, 'dominant': 24, 'aroma': 25, 'sweet': 26, 'it': 27, 'cross': 28, 'are': 29, 'body': 30, 'genetics': 31, 'high': 32, 'purple': 33, 'has': 34, 'buds': 35, 'sour': 36, 'on': 37, 'be': 38, 'pain': 39, 'you': 40, 'but': 41, 'bred': 42, 'while': 43, 'haze': 44, 'seeds': 45, 'diesel': 46, 'cannabis': 47, 'or': 48, 'between': 49, 'white': 50, 'thc': 51, 'blue': 52, 'potent': 53, 'flavor': 54, 'green': 55, 'will': 56, 'known': 57, 'earthy': 58, 'can': 59, 'stress': 60, 'heavy': 61, 'skunk': 62, 'citrus': 63, 'flowering': 64, 'like': 65, 'lemon': 66, 'flavors': 67, 'have': 68, 'strains': 69, 'strong': 70, 'cerebral': 71, 'relaxing': 72, 'at': 73, 'was': 74, 'uplifting': 75, 'mind': 76, 'also': 77, 'blueberry': 78, 'consumers': 79, 'buzz': 80, '

In [105]:
print(t.word_docs)

defaultdict(<class 'int'>, {'alenuihaha': 2, 'is': 3487, 'a': 5914, 'sativa': 1410, 'dominant': 854, 'hybrid': 2246, 'by': 1392, 'pua': 12, 'mana': 11, '1st': 54, 'hawaiian': 117, 'pakalolo': 10, 'seed': 130, 'bank': 31, 'named': 150, 'after': 240, 'the': 5317, 'sea': 13, 'channel': 1, 'that': 2059, 'separates': 1, 'kona': 14, 'side': 69, 'of': 4215, 'big': 90, 'island': 38, 'and': 6641, 'maui': 48, 'blending': 10, 'gold': 84, 'wowie': 14, 'achieves': 7, 'an': 1158, 'uplifted': 30, 'happy': 127, 'stimulation': 54, "doesn't": 14, 'overclock': 1, "consumer's": 14, 'heart': 13, 'or': 420, 'mind': 294, 'this': 2189, "strain's": 162, 'healthy': 16, 'yield': 82, 'consistently': 9, 'positive': 15, 'effects': 1327, 'make': 159, 'it': 758, 'delightful': 8, 'inspiring': 7, 'choice': 152, 'for': 1631, 'consumers': 276, 'seeking': 57, 'relief': 196, 'from': 947, 'depression': 150, 'fatigue': 43, '100': 7, 'og': 1149, '50': 115, 'strain': 2218, 'packs': 17, 'strong': 314, 'punch': 68, 'name': 183, 

##Start of King Burrito Set

In [0]:
df_burrito = pd.read_csv("https://raw.githubusercontent.com/bw-ft-medcab3-brian/ds/master/data/source/cannabis.csv")

In [107]:
df_burrito.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [108]:
len(df_kushy)

2130

In [109]:
len(df_burrito)

2351

In [0]:
df_burrito = df_burrito.drop(["Rating"], axis=1)

In [0]:
burrito_string = []

In [0]:
for each in range(2350):
  burrito_string = burrito_string + text_to_word_sequence(str(df_burrito['Effects'][each]))
  burrito_string = burrito_string + text_to_word_sequence(str(df_burrito['Flavor'][each]))
  burrito_string = burrito_string + text_to_word_sequence(str(df_burrito['Description'][each]))
  burrito_string = burrito_string + text_to_word_sequence(str(df_burrito['Strain'][each]))
  burrito_string = burrito_string + text_to_word_sequence(str(df_burrito['Type'][each]))

In [0]:
tk = Tokenizer()
tk.fit_on_texts(burrito_string)

In [114]:
print(tk.word_counts)

OrderedDict([('creative', 892), ('energetic', 736), ('tingly', 359), ('euphoric', 1819), ('relaxed', 1805), ('earthy', 1482), ('sweet', 1894), ('citrus', 889), ('100', 8), ('og', 1224), ('is', 3769), ('a', 6366), ('50', 130), ('hybrid', 2432), ('strain', 2414), ('that', 2236), ('packs', 18), ('strong', 334), ('punch', 65), ('the', 5860), ('name', 196), ('supposedly', 18), ('refers', 17), ('to', 2958), ('both', 194), ('its', 1651), ('strength', 14), ('and', 7208), ('high', 626), ('price', 2), ('when', 194), ('it', 816), ('first', 106), ('started', 5), ('showing', 4), ('up', 178), ('in', 1825), ('hollywood', 5), ('as', 944), ('plant', 247), ('tends', 39), ('produce', 101), ('large', 125), ('dark', 114), ('green', 381), ('buds', 600), ('with', 2893), ('few', 37), ('stems', 10), ('users', 84), ('report', 19), ('body', 743), ('effect', 93), ('of', 4552), ('an', 1239), ('indica', 2044), ('for', 1752), ('pain', 514), ('relief', 210), ('more', 184), ('alert', 30), ('cerebral', 332), ('feeling'

In [115]:
print(tk.document_count)

200505


In [116]:
print(tk.word_index)

{'and': 1, 'a': 2, 'the': 3, 'of': 4, 'is': 5, 'to': 6, 'with': 7, 'hybrid': 8, 'this': 9, 'strain': 10, 'that': 11, 'indica': 12, 'happy': 13, 'sweet': 14, 'in': 15, 'euphoric': 16, 'relaxed': 17, 'for': 18, 'its': 19, 'by': 20, 'uplifted': 21, 'effects': 22, 'sativa': 23, 'earthy': 24, 'an': 25, 'og': 26, 'kush': 27, '\xa0': 28, 'from': 29, 'as': 30, 'dominant': 31, 'creative': 32, 'citrus': 33, 'aroma': 34, 'it': 35, 'sleepy': 36, 'cross': 37, 'body': 38, 'energetic': 39, 'pungent': 40, 'are': 41, 'diesel': 42, 'genetics': 43, 'focused': 44, 'purple': 45, 'high': 46, 'has': 47, 'buds': 48, 'on': 49, 'berry': 50, 'pine': 51, 'sour': 52, 'skunk': 53, 'while': 54, 'haze': 55, 'lemon': 56, 'pain': 57, 'be': 58, 'but': 59, 'seeds': 60, 'you': 61, 'hungry': 62, 'bred': 63, 'cannabis': 64, 'or': 65, 'thc': 66, 'white': 67, 'blueberry': 68, 'between': 69, 'potent': 70, 'blue': 71, 'flavor': 72, 'known': 73, 'green': 74, 'talkative': 75, 'stress': 76, 'will': 77, 'tingly': 78, 'like': 79, 'c

In [117]:
print(tk.word_docs)

defaultdict(<class 'int'>, {'creative': 892, 'energetic': 736, 'tingly': 359, 'euphoric': 1819, 'relaxed': 1805, 'earthy': 1482, 'sweet': 1894, 'citrus': 889, '100': 8, 'og': 1224, 'is': 3769, 'a': 6366, '50': 130, 'hybrid': 2432, 'strain': 2414, 'that': 2236, 'packs': 18, 'strong': 334, 'punch': 65, 'the': 5860, 'name': 196, 'supposedly': 18, 'refers': 17, 'to': 2958, 'both': 194, 'its': 1651, 'strength': 14, 'and': 7208, 'high': 626, 'price': 2, 'when': 194, 'it': 816, 'first': 106, 'started': 5, 'showing': 4, 'up': 178, 'in': 1825, 'hollywood': 5, 'as': 944, 'plant': 247, 'tends': 39, 'produce': 101, 'large': 125, 'dark': 114, 'green': 381, 'buds': 600, 'with': 2893, 'few': 37, 'stems': 10, 'users': 84, 'report': 19, 'body': 743, 'effect': 93, 'of': 4552, 'an': 1239, 'indica': 2044, 'for': 1752, 'pain': 514, 'relief': 210, 'more': 184, 'alert': 30, 'cerebral': 332, 'feeling': 77, 'thanks': 27, 'sativa': 1483, 'side': 72, 'aroused': 199, 'happy': 1994, 'flowery': 276, 'violet': 22, '